<a href="https://colab.research.google.com/github/hario27/PositionChecker/blob/master/keras1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperas

In [2]:
!pip install tsfresh

     |████████████████████████████████| 102kB 5.9MB/s 
     |████████████████████████████████| 686kB 18.5MB/s 
     |████████████████████████████████| 1.1MB 31.0MB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 1.3MB 50.8MB/s 
     |████████████████████████████████| 112kB 54.2MB/s 
ERROR: googleapis-common-protos 1.53.0 has requirement protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.
ERROR: google-api-core 1.26.1 has requirement protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.
ERROR: distributed 2021.3.1 has requirement dask>=2021.03.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: stumpy 1.8.0 has requirement scipy>=1.5, but you'll have scipy 1.4.1 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: distributed 1.25.3
    Uninstalling distri

In [3]:
import pandas as pd
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

#from tsfresh import extract_features
#from tsfresh import select_features

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import Lasso, LassoCV

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

Mounted at /content/drive


In [4]:
def data():
  # CSV読み込み
  print("CSV読み込み==============================================================================")
  df_origin = pd.read_csv("/content/drive/My Drive/Colab Notebooks/feature_data_test.csv")

  # 特徴量作成
  """
  print("\n特徴量作成==============================================================================")
  temp = pd.DataFrame(df["Close"], columns=['Close'], dtype='float')
  serial_num = pd.RangeIndex(start=0, stop=len(df.index) , step=1)
  temp["id"] = serial_num
  extracted_features = extract_features(temp, column_id='id')
  print(extracted_features)
  df = pd.concat([df, extracted_features], axis=1)
  df = df.dropna(axis=1)
  """

  # データ分割
  print("\n訓練テストデータ分割==============================================================================")
  df = df_origin[0:len(df_origin) - 30]
  print(df)
  df = df[-5000:]
  df.reset_index(inplace=True)
  X = df.drop(["datetime", "Target(30)", "Open", "Close", "High", "Low", "index", "Volume", "hour", "dayweek"], axis=1)
  y = df[f"Target({str(30)})"]
  print(X)

  print("\n検証データ分割==============================================================================")
  df1 = df_origin[-30:]
  df1.reset_index(inplace=True)
  X1 = df1.drop(["datetime", "Target(30)", "Open", "Close", "High", "Low", "index", "Volume", "hour", "dayweek"], axis=1)
  print(X1)
  
  # 次元削減
  """
  print("\n次元削減==============================================================================")
  X_selected = select_features(X, y)
  X_selected["hour"] = hour
  X_selected["dayweek"] = dayweek
  print(X_selected)
  """

  # 次元圧縮
  """
  print("\n次元圧縮==============================================================================")
  n_components = int(X_selected.shape[1] * 0.8)
  pca = PCA(n_components=n_components)
  pca.fit(X_selected)
  X_selected_pca = pca.transform(X_selected)
  print(X_selected_pca.shape)
  print(X_selected_pca)
  """

  # ホールドアウト分割
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

  """
  print("SelectPercentile")
  select = SelectPercentile(percentile=5)
  print("FIT")
  select.fit(x_train, y_train)
  """

  """
  criterion="gini"
  numFeatures = 6
  nEstimators = 65
  predWindow = 1
  oobScore = True
  randomForest1 = RandomForestClassifier(
      n_estimators = nEstimators,
      max_features = numFeatures,
      bootstrap = True,
      oob_score = oobScore,
      verbose = 0,
      criterion = criterion,
      n_jobs = -1,
      random_state = 42
  )
  """

  """
  print("SelectFromModel")
  select = SelectFromModel(estimator=randomForest1, threshold=0.001)
  print("FIT")
  select.fit(x_train, y_train)
  """

  print("RFE")
	#select = RFECV(lgb.LGBMClassifier(objective="binary", metric="binary_logloss", random_state=42), step=1, scoring="accuracy", verbose=1)
  #select = RFE(lgb.LGBMClassifier(objective="binary", logloss="rmse", random_state=42), n_features_to_select=80, step=0.5, verbose=1)
	#clf = Lasso(alpha=0.1, max_iter=1000)
  clf = RandomForestClassifier(n_estimators=100, random_state=42)
  select = RFE(estimator=clf, n_features_to_select=40, verbose=1)
  print("FIT")
  select.fit(x_train, y_train) 
  display(x_train.columns.values[select.support_])

  print(x_train.shape)
  x_train = select.transform(x_train)
  x_test = select.transform(x_test)
  X1_selected_pca = select.transform(X1)
  print(x_train.shape)
  print(X1_selected_pca.shape)

  """
  print("aaaa")
  print(x_train)
  x_train = x_train[['slowd17', 'slowd20', 'slowd71', 'slowd72', 'slowd88', 'slowk91',
       'slowk100', 'slowd104', 'slowd123', 'slowd141', 'PPO149', 'PPO181',
       'PPO195', 'slowk197', 'slowd200', 'PPO203', 'slowd203', 'slowd210',
       'detrendedClose', 'ADX25', 'WR65', 'WR75', 'ADX135',
       'lowerband145', 'CORREL145', 'CORREL155', 'kairi165', 'CORREL165',
       'upperband175', 'EMA185', 'lowerband195', 'EMA195', 'WR195',
       'PROC205', 'ADX215', 'CCI215', 'PROC215', 'TYPPRICE', 'WCLPRICE',
       'AD']]
  x_test = x_test[['slowd17', 'slowd20', 'slowd71', 'slowd72', 'slowd88', 'slowk91',
       'slowk100', 'slowd104', 'slowd123', 'slowd141', 'PPO149', 'PPO181',
       'PPO195', 'slowk197', 'slowd200', 'PPO203', 'slowd203', 'slowd210',
       'detrendedClose', 'ADX25', 'WR65', 'WR75', 'ADX135',
       'lowerband145', 'CORREL145', 'CORREL155', 'kairi165', 'CORREL165',
       'upperband175', 'EMA185', 'lowerband195', 'EMA195', 'WR195',
       'PROC205', 'ADX215', 'CCI215', 'PROC215', 'TYPPRICE', 'WCLPRICE',
       'AD']]

  X1_selected_pca = X1
  X1_selected_pca = X1_selected_pca[['slowd17', 'slowd20', 'slowd71', 'slowd72', 'slowd88', 'slowk91',
       'slowk100', 'slowd104', 'slowd123', 'slowd141', 'PPO149', 'PPO181',
       'PPO195', 'slowk197', 'slowd200', 'PPO203', 'slowd203', 'slowd210',
       'detrendedClose', 'ADX25', 'WR65', 'WR75', 'ADX135',
       'lowerband145', 'CORREL145', 'CORREL155', 'kairi165', 'CORREL165',
       'upperband175', 'EMA185', 'lowerband195', 'EMA195', 'WR195',
       'PROC205', 'ADX215', 'CCI215', 'PROC215', 'TYPPRICE', 'WCLPRICE',
       'AD']]
  """

  # 正規化
  sc = StandardScaler()
  sc.fit(x_train)
  x_train = sc.transform(x_train)
  x_test = sc.transform(x_test)
  x_valid = sc.transform(X1_selected_pca)

  display(x_test.shape)

  """
  poly = PolynomialFeatures(degree=2).fit(x_train)
  x_train = poly.transform(x_train)
  x_test = poly.transform(x_test)
  """

  x_train = np.asarray(x_train).astype("float64")
  x_test = np.asarray(x_test).astype("float64")
  y_train = np.asarray(y_train).astype("float32")
  y_test = np.asarray(y_test).astype("float32")
  x_valid = np.asarray(x_valid).astype("float32")

  return x_train, y_train, x_test, y_test, x_valid

In [5]:
def model(x_train, y_train, x_test, y_test, x_valid):
  model = Sequential()
  model.add(Dense({{choice([32, 64, 128, 256, 512, 1024])}}, activation={{choice(["relu", "tanh"])}}, input_shape=(x_train.shape[1],)))
  model.add(Dropout({{uniform(0, 1)}}))
  model.add(Dense({{choice([32, 64, 128, 256, 512, 1024])}}, activation={{choice(["relu", "tanh"])}}))
  model.add(Dropout({{uniform(0, 1)}}))
  model.add(Dense(1, activation="sigmoid"))
  sgd = SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
  model.compile(optimizer=sgd, loss="mse", metrics=["acc"])
  callbacks_list = [EarlyStopping(monitor="val_loss", patience=10)]
  history = model.fit(x_train, y_train, epochs=1000, batch_size={{choice([32, 64, 126, 256, 512])}}, validation_data=(x_test, y_test), verbose=0, callbacks=callbacks_list)

  val_loss, val_acc = model.evaluate(x_test, y_test, verbose=0)
  print('Test accuracy:', val_acc)
  print('Test loss:', val_loss)

  """
  history_dict = history.history
  loss_values = history_dict["loss"]
  val_loss_values = history_dict["val_loss"]

  epochs = range(1, len(loss_values) + 1)
  
  plt.clf()
  # "bo"は"blue dot(青のドット)を意味する
  plt.plot(epochs, loss_values, "b", label="Traning loss")
  # "b"はsolod blue line(青の実線)を意味する
  plt.plot(epochs, val_loss_values, "r", label="Validation loss")
  plt.title("Traning and validation loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()
  """

  return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=1000,
                                          notebook_name=os.path.join('drive','My Drive','Colab Notebooks','keras1'),
                                          trials=Trials())

print(best_model.summary())
print(best_run)

_, _, x_test, y_test, x_valid = data()
print("tesuto")
print(x_valid)
val_loss, val_acc = best_model.evaluate(x_test, y_test)
print("val_loss: ", val_loss)
print("val_acc: ", val_acc)
y_pred_proba = best_model.predict(x_valid)
y_pred = np.where(y_pred_proba > 0.5, 1, 0)
print(y_pred)

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import os
except:
    pass

try:
    from google.colab import drive
except:
    pass

try:
    from sklearn.decomposition import PCA
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler, PolynomialFeatures
except:
    pass

try:
    from sklearn.feature_selection import SelectPercentile
except:
    pass

try:
    from sklearn.feature_selection import SelectFromModel
except:
    pass

try:
    from sklearn.feature_selection import RFECV, RFE
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV, cross_val_score
except:
    pass

try:
    from sklearn.ensemble import RandomForestClassifier
except:
    pass

try:
    import lightgbm as lgb
except:
    pass

try:
    import xgboost as xgb
except:
    pass

try:
    from sklearn.linear_mode

In [ ]:
y_pred_proba = best_model.predict(x_test)
y_pred = np.where(y_pred_proba > 0.5, 1, 0)
print("CSV読み込み==============================================================================")
df1 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/feature_data_test.csv")
df1 = df1[0:len(df1) - 30]
display(df1)
df1 = df1[-5000:]
df1 = df1.reset_index()
df1 = df1[-500:]
df1["target"] = y_pred
display(df1)
pd.DataFrame(df1).to_csv("/content/drive/My Drive/Colab Notebooks/keras_pre.csv")

In [ ]:
#best_model.save("/content/drive/My Drive/Colab Notebooks/my_model.h5")

In [ ]:
#new_model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/my_model.h5")
#print(new_model.summary())

In [ ]:
"""
sum(df1["Target(30)"] == df1["target"])
print(sum(df1["Target(30)"] == df1["target"]) / len(df1["Target(30)"]))
"""

In [ ]:
"""
model = Sequential()
model.add(Dense(384, activation="tanh", input_shape=(x_train.shape[1],)))
model.add(Dropout(0.9633154977822115))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.7168429952431055))
model.add(Dense(384, activation="tanh"))
model.add(Dropout(0.845687831736537))
model.add(Dense(1, activation="sigmoid"))
sgd = SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(optimizer="adamax", loss="mse", metrics=["acc"])
"""
# hourないver
"""
model = Sequential()
model.add(Dense(768, activation="tanh", input_shape=(x_train.shape[1],)))
model.add(Dropout(0.990002599701736))
model.add(Dense(384, activation="tanh"))
model.add(Dropout(0.39853942914582385))
model.add(Dense(128, activation="tanh"))
model.add(Dropout(0.9172213806474925))
model.add(Dense(1, activation="sigmoid"))
sgd = SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(optimizer="adamax", loss="mse", metrics=["acc"])
callbacks_list = [EarlyStopping(monitor="val_loss", patience=10)]
history = model.fit(x_train, y_train, epochs=1000, batch_size=512, validation_data=(x_test, y_test), verbose=0, callbacks=callbacks_list)
"""

In [ ]:
"""
print(x_train.shape)
print(y_train.shape)
x_train1 = x_train.reshape(-1, 1, 259)
x_test1  = x_test.reshape(-1, 1, 259)
y_train1 = y_train.reshape(-1, 1, 1)
y_test1 = y_test.reshape(-1, 1, 1)

model = keras.Sequential([
    keras.layers.LSTM(512,input_shape=(1, 259),
                          return_sequences = True),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(30,
            return_sequences=False),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1,activation='sigmoid')
])
"""
"""
x_train, y_train, x_test, y_test = data()
model = Sequential()
model.add(Dense(1024, activation="relu", input_shape=(x_train.shape[1],)))
model.add(Dropout(0.6863068699311096))
model.add(Dense(1024, activation="tanh"))
model.add(Dropout(0.7534624203389924))
model.add(Dense(1, activation="sigmoid"))
sgd = SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['acc'])
callbacks_list = [EarlyStopping(monitor="val_loss", patience=10)]
history = model.fit(x_train, y_train, epochs=1000, batch_size=512, validation_data=(x_test, y_test), verbose=0, callbacks=callbacks_list)
val_loss, val_acc = model.evaluate(x_test1, y_test1, verbose=0)
print('Test accuracy:', val_acc)
print('Test loss:', val_loss)
print(model.summary())
y_pred_proba = model.predict(x_test)
#print(y_pred_proba)
#y_pred = np.where(y_pred_proba > 0.5, 1, 0)
#print(y_pred)
#pd.DataFrame(y_pred).to_csv("/content/drive/My Drive/Colab Notebooks/data5.csv")
"""